# 레퍼런스의 레퍼런스로 문장 간 유사도 측정

In [ ]:
# ref) https://github.com/jhgan00/ko-sentence-transformers?tab=readme-ov-file
# ref) https://huggingface.co/jhgan/ko-sroberta-multitask


In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

In [25]:
corpus = ['드가자! 신 혁명파 행동대장 100:1의 전설.',
          '<연인> 최종회 Part.2 새로운 여주인공!!',
          '백성을 훔친 도적 역적 결말까지 몰아보기',
          '《1시간 순삭!!》 "더글로리"  "임지연"의 명연기 한국 범죄 수사 드라마 몰아보기!!',
          '[환혼: 빛과 그림자] 2화 예고 / 안죽은거야?! 진짜 진부연이 나타난다!',
          '지옥 끝까지 찾아가 연진이에게 복수하는 동은 - 더글로리',
          '[갯마을 초등학교 학예회] 3학년 5반 - 홍박사님을 아세요?',
          '갯마을 차차차 1화 ~ 10화 한번에 몰아보기',
          '갯마을고 학생들의 홍박사 챌린지',
           '"더글로리"보다 더 재밌는 임지연의 신작 드라마',
           '메이플스토리 도적 육성 가이드 완결',
          '도적의 대신 돈 백성의 훔친 운명은 마음을?',
          '돈 신대 의성백 의도적 친 훔 은운명 음을마 ?']

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [26]:
corpus_embeddings.shape

torch.Size([13, 768])

In [27]:
# Query sentences:
queries = ['돈 대신 백성의 마음을 훔친 도적의 운명은?',
           '[더글로리 10화] 박연진의 최후는?',
           '[갯마을 차차차] 홍반장의 시점으로 본 갯마을 차차차']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 3
for query in queries:
  query_embedding = embedder.encode(query, convert_to_tensor=True)
  cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
  cos_scores = cos_scores.cpu()

  #We use np.argpartition, to only partially sort the top_k results
  top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

  print("\n\n======================\n\n")
  print("Query:", query)
  print(f"\nTop {top_k} most similar sentences in corpus:")

  i = 1
  for idx in top_results[0:top_k]:
    print(f'{i}.', corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
    i += 1





Query: 돈 대신 백성의 마음을 훔친 도적의 운명은?

Top 3 most similar sentences in corpus:
1. 도적의 대신 돈 백성의 훔친 운명은 마음을? (Score: 0.8613)
2. 백성을 훔친 도적 역적 결말까지 몰아보기 (Score: 0.6760)
3. 돈 신대 의성백 의도적 친 훔 은운명 음을마 ? (Score: 0.4122)




Query: [더글로리 10화] 박연진의 최후는?

Top 3 most similar sentences in corpus:
1. 지옥 끝까지 찾아가 연진이에게 복수하는 동은 - 더글로리 (Score: 0.4348)
2. 《1시간 순삭!!》 "더글로리"  "임지연"의 명연기 한국 범죄 수사 드라마 몰아보기!! (Score: 0.4080)
3. 갯마을 차차차 1화 ~ 10화 한번에 몰아보기 (Score: 0.3839)




Query: [갯마을 차차차] 홍반장의 시점으로 본 갯마을 차차차

Top 3 most similar sentences in corpus:
1. 갯마을 차차차 1화 ~ 10화 한번에 몰아보기 (Score: 0.6217)
2. [갯마을 초등학교 학예회] 3학년 5반 - 홍박사님을 아세요? (Score: 0.4851)
3. 갯마을고 학생들의 홍박사 챌린지 (Score: 0.4613)


# KoBERT 해결!!

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install transformers
!pip install torch

In [2]:
!pip install sentencepiece==0.1.96

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.5 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 428 (delta 129), reused 108 (delta 106), pack-reused 275
Receiving objects: 100% (428/428), 219.68 KiB | 6.10 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [4]:
%cd /content/KoBERT

/content/KoBERT


# 여기서 requirements.txt 파일을 설치하지 마세요

In [ ]:
!pip install boto3
!pip install gluonnlp
!pip install mxnet
!pip install onnxruntime
!pip install torch
!pip install transformers

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [8]:
#KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
#bertmodel의 vocabulary
bertmodel, vocab = get_pytorch_kobert_model()

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
#GPU 설정
device = torch.device("cuda:0")

In [11]:
data_list = [
    ['저러니까 자신보다 어린 사람한테 미개하다는 소리듣지', '6'],
['컴백까지 건강챙기시고 멋있는모습으로 만나요?', '5'],
['친구들이 나를 힘들게 해서 전학을 가게 되었어. 새로운 곳에서는 천천히 친구들을 사귀어보고 싶어.', '5'],
['나는 이직한 지금 회사에 불만이 없어.', '5'],
['자꾸만 화가나고 욕을하게됩니다', '3'],
['나 너무 우울하고 외로워.', '3'],
['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '0']
]
data_list

[['저러니까 자신보다 어린 사람한테 미개하다는 소리듣지', '6'],
 ['컴백까지 건강챙기시고 멋있는모습으로 만나요?', '5'],
 ['친구들이 나를 힘들게 해서 전학을 가게 되었어. 새로운 곳에서는 천천히 친구들을 사귀어보고 싶어.', '5'],
 ['나는 이직한 지금 회사에 불만이 없어.', '5'],
 ['자꾸만 화가나고 욕을하게됩니다', '3'],
 ['나 너무 우울하고 외로워.', '3'],
 ['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '0']]

In [12]:
for i in range(10):
  data_list.extend(data_list)
print("Done!!!")
len(data_list)

Done!!!


7168

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
#tokenize
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
#data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

#train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
#test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
#data_train = BERTDataset(data_list, 0, 1, tok, max_len, True, False)
# /usr/local/lib/python3.10/dist-packages/sentencepiece/__init__.py i
# -> nbest_size = 2
data_train = BERTDataset(data_list, 0, 1, tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


<ipython-input-20-957c024024d2>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/112 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.020480155944824 train acc 0.15625
epoch 1 train acc 0.8687220982142857


  0%|          | 0/112 [00:00<?, ?it/s]

...

epoch 3 batch id 1 loss 0.0036995401605963707 train acc 1.0
epoch 3 train acc 1.0


  0%|          | 0/112 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.0023213380482047796 train acc 1.0
epoch 4 train acc 1.0


  0%|          | 0/112 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0021305610425770283 train acc 1.0
epoch 5 train acc 1.0


In [21]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    #model1.eval()
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        #out = model1(token_ids, valid_length, segment_ids)
        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(test_eval[0] + " 느껴집니다.")

using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [22]:
predict("우리나라 아름답다")

슬픔이 느껴집니다.


In [24]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 하이
슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


In [ ]:
# ref : https://velog.io/@fhflwhwl5/Python-KoBERT-7%EA%B0%80%EC%A7%80-%EA%B0%90%EC%A0%95%EC%9D%98-%EB%8B%A4%EC%A4%91%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0